# Pandas 2

## Introducción


En esta notebook vamos a trabajar con los conceptos de agregación a través de `groupby` y `pivot tables`. 

Analizaremos características de precio y superficie, por barrio y por comuna, de inmuebles en venta en la Ciudad de Buenos Aires en el año 2016.


## Dataset

El Gobierno de la Ciudad de Buenos Aires disponibiliza algunos datasets para acceso público en la url 
https://data.buenosaires.gob.ar/

Allí encontramos datos de propiedades inmuebles en venta, separados por año
https://data.buenosaires.gob.ar/dataset/departamentos-venta

En esta clase vamos a usar un subconjunto de los datos de "Departamentos en venta 2016" que descargamos desde aquí
https://data.buenosaires.gob.ar/dataset/departamentos-venta/archivo/juqdkmgo-7031-resource

En la carpeta Data de esta clase ya tenemos descargardo ese dataset, el nombre del archivo es **departamentos-en-venta-2016.csv**.

## Imports

In [1]:
import pandas as pd
import numpy as np

## Ejercicio 1  - Importar 

Leamos los datos del archivo departamentos-en-venta-2016.csv

Veamos cuántos registros tiene y de qué tipos son los datos de cada columna. 

Veamos los primeros registros para verificar que los datos fueron importados correctamente.

In [2]:
import chardet
def get_encoding_type(csv_path):
    rawdata = open(csv_path, 'rb').read()
    result = chardet.detect(rawdata)
    return result.get('encoding')

In [3]:
data_location = "../Data/departamentos-en-venta-2016.csv"

In [8]:
get_encoding_type(data_location)

'utf-8'

In [12]:
df = pd.read_csv(data_location, sep=';', encoding='utf-8')

In [51]:
df.tail()

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,URL,REVISION,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD
7559,NaN,NaN,42302009,VTA,DTO,37,32,U$S 84.700,1482250,0,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42302...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7560,NaN,NaN,42302010,VTA,DTO,44,44,U$S 99.000,1732500,0,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42302...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7561,NaN,NaN,42302011,VTA,DTO,45,40,U$S 95.220,1666350,0,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42302...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7562,NaN,NaN,42302012,VTA,DTO,44,39,U$S 95.220,1666350,0,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42302...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7563,"GARAY, JUAN DE AV.",612.0,42394651,VTA,DTO,51,46,U$S 147.619,2583332,0,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42394...,NaN,NaN,"GARAY, JUAN DE AV. 612",SAN TELMO,COMUNA 01,1153.0,C1153ABR,-34.624541,-58.374337


In [14]:
df.shape

(7564, 29)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7564 entries, 0 to 7563
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CALLE                    6182 non-null   object 
 1   NUMERO                   6182 non-null   float64
 2   ID_ZONAPRO               7564 non-null   int64  
 3   OPERACION                7564 non-null   object 
 4   TIPO                     7564 non-null   object 
 5   M2                       7564 non-null   int64  
 6   M2CUB                    7564 non-null   int64  
 7   PRECIOTEXT               7562 non-null   object 
 8   PRECIOARS                7564 non-null   int64  
 9   PRECIOARSM               7564 non-null   int64  
 10  DOLARES                  7564 non-null   int64  
 11  U_S_M2                   7564 non-null   int64  
 12  AMBIENTES                7564 non-null   int64  
 13  ANTIGUEDAD               7564 non-null   int64  
 14  BAÑOS                   

## Ejercicio 2  - Promedio

¿Cuál es el promedio valor del precio por $m^2$ en dolares?

Para responder esta pregunta vamos a:

1) Crear un obeto Series que resulte de limpiar los valores del campo PRECIOTEXT. Tenemos que eliminar los símbolos "U\\$S" y "."

2) En la instancia de Series resultado del punto 1) reemplazamos los valores en pesos ('\\$') por nulos (para que nos ensucien los datos de promedio). 

3) Crear una columna nueva de tipo numérico en el DataFrame (PRECIOTEXT_CLEAN) y asignar los valores de resultado de 2)

3) Crear una columna de tipo numérico (PRECIOTEXTM) que tenga el precio del m2 en dolares calculado como el valor de PRECIOTEXT_CLEAN / M2

4) Calcular la media de los valores de PRECIOTEXTM


In [106]:
def limpiar_precio(precio):
    precio = str(precio)
    valor = precio
    # Está en dólares?
    if precio.startswith('U$S '):
        valor = precio.lstrip('U$S ')
    # Remuevo el punto separador de miles
    if '.' in precio:
        valor = valor.replace('.','')
    # Elimino los valores en pesos
    if precio.startswith('$'):
        valor = None
    return valor

In [107]:
df['PRECIOTEXT_CLEAN'] = df['PRECIOTEXT'].apply(limpiar_precio).astype('float')

In [110]:
df['PRECIOTEXTM'] = df['PRECIOTEXT_CLEAN']/df['M2']

In [116]:
print('Promedio de valor en dólares del M2: ')
df['PRECIOTEXTM'].mean().round(2)

Promedio de valor en dólares del M2: 


2522.49

## Ejercicio 3  - Promedio por cuartil
¿Cuál es el precio promedio del metro cuadrado en dolares para cada cuartil de superficie (campo M2) de las viviendas en CABA?

Comenzamos calculando los cuartilos de superficie.

In [132]:
q1_m2 = df['M2'].quantile(0.25)
q2_m2 = df['M2'].quantile(0.50)
q3_m2 = df['M2'].quantile(0.75)
q4_m2 = df['M2'].quantile(1)
q1_m2, q2_m2, q3_m2, q4_m2

(41.0, 54.0, 80.0, 730.0)

In [136]:
df_q1 = df[df['M2']<=q1_m2]
df_q2 = df[(df['M2']>q1_m2) & (df['M2']<=q2_m2)]
df_q3 = df[(df['M2']>q2_m2) & (df['M2']<=q3_m2)]
df_q4 = df[df['M2']>q3_m2]

In [143]:
df_q1['PRECIOTEXTM'].mean().round(2), df_q2['PRECIOTEXTM'].mean().round(2), df_q3['PRECIOTEXTM'].mean().round(2), df_q4['PRECIOTEXTM'].mean().round(2)

(2418.49, 2455.99, 2438.61, 2788.3)

In [149]:
# Forma alternativa con qcut y groupby
df_qs = pd.qcut(df['M2'],4)
df.groupby(df_qs)['PRECIOTEXTM'].mean().round(2)

M2
(14.999, 41.0]    2418.49
(41.0, 54.0]      2455.99
(54.0, 80.0]      2438.61
(80.0, 730.0]     2788.30
Name: PRECIOTEXTM, dtype: float64

## Ejercicio 3  - Promedio por barrio

¿Cuál es la media de precio por metro cuadrado en dolares para cada barrio de la CABA? 

Ordenemos los datos para indicar cuál es el barrio más caro.

Resolvamos el cálculo tanto con groupby como con pivot tables

In [161]:
df.groupby('BARRIO')['PRECIOTEXTM']\
            .mean()\
            .sort_values(ascending=False)

BARRIO
PUERTO MADERO        5412.468738
PALERMO              3136.014984
BELGRANO             3110.065913
RETIRO               2941.052687
RECOLETA             2876.201544
PARQUE CHACABUCO     2645.412539
COLEGIALES           2636.723717
NUÑEZ                2597.472491
COGHLAN              2451.120924
CABALLITO            2412.777566
VILLA ORTUZAR        2408.765643
VILLA URQUIZA        2402.505658
SAAVEDRA             2365.061414
MONSERRAT            2349.127916
CHACARITA            2338.939470
VILLA CRESPO         2331.951318
AGRONOMIA            2276.703779
ALMAGRO              2252.269793
VILLA DEL PARQUE     2240.717867
BOEDO                2215.391462
CONSTITUCION         2197.241584
VILLA DEVOTO         2190.415302
SAN NICOLAS          2186.049883
BARRACAS             2166.146707
VILLA PUEYRREDON     2159.102203
SAN TELMO            2145.817227
PARQUE CHAS          2137.475822
BALVANERA            1988.284935
BOCA                 1963.575157
VILLA LURO           1960.143435
MON

In [160]:
pd.pivot_table(df, values='PRECIOTEXTM', index='BARRIO')\
                .sort_values(by='PRECIOTEXTM', ascending=False)

,PRECIOTEXTM
BARRIO,
PUERTO MADERO,5412.468738
PALERMO,3136.014984
BELGRANO,3110.065913
RETIRO,2941.052687
RECOLETA,2876.201544
PARQUE CHACABUCO,2645.412539
COLEGIALES,2636.723717
NUÑEZ,2597.472491
COGHLAN,2451.120924


## Ejercicio 4  - Dispersión de precios por barrio

**4.a** ¿En qué barrio hay una mayor dispersión en el valor del metro cuadrado en dolares? Ordenar los valores para identificar el mayor.

Ayuda: Calculamos dispersión como el desvío estandar de un grupo dividido la media de ese grupo grupo

Vamos a calcular esto de dos maneras distintas:

1) Calculamos la instancia de Series que tiene la media por grupo. Calculamos la instancia de Series que tiene el desvío estandar por grupo. Las dividimos. Ordenamos

2) Calculamos los grupos y usamos una función lambda que calcule la media, desvío estandar y divida. Ordenamos.

**4.b** Cuál les parece que será más eficiente? Probemoslo con %timeit, para eso definamos dos funciones que encapsulen el código de 1) y 2).

**4.c** ¿Por qué Paternal está devolviendo NaN? ¿Y por qué Villa Soldati devuelve 0?

Nota: tengan en cuenta que `mean` y `std` sobre una instancia de `DataFrameGroupBy` excluye los valores nulos

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.mean.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html





Vamos a ver cuánto da si usamos la lambda directamente, sin pasar por dispersion_2:

Veamos por qué Paternal devuelve NaN y Villa Soldati que devuelve 0.


## Ejercicio 5  - Superficie por barrio

Calculemos la mediana de superficie por barrio para determinar en qué barrio los departamentos son más grandes.

Hagamos el cálculo con pivot_table y groupby


## Ejercicio 6  - Tabla resumen

Generemos un `DataFrame` que agregue la información del precio por M2 en dolares (`PRECIOTEXTM`), ambientes (`AMBIENTES`) a nivel de `COMUNA` y barrio (`BARRIO`). Proporcione información tanto de la tendencia central como de la dispersión de ambas distribuciones.

Ayuda: usar pivot_table